In [1]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
import SimpleITK as sitk
import pydicom as dicom
from pydicom.tag import Tag
import sys
import csv
import pydicom as pdcm

In [16]:
def getICUID(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            slices = pdcm.read_file(root+f)
            return slices.file_meta.ImplementationClassUID

In [18]:
window_type = 'lung'
root_path="//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/"
structured = root_path + "DATA_VOLLEDIG_structured/"
unstructured = root_path + "DATA_VOLLEDIG_unstructured/"
csv_filename = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/ListPETMatchACCT.csv"

image_generation_flag=True
mask_generation_flag=True

#Folders to look at:
Try1 ="CT_DICOM_CLEAN/"
Try2 ="PET_DICOM/"
#Try3 ="PET_LDCT_REGISTERED/"
Try4 ="PET_MIRADA_REGISTERED/"

In [14]:
MergeLists=[]
MergeLists.append(os.path.join(structured,Try1))
MergeLists.append(os.path.join(structured,Try2))
#MergeLists.append(os.path.join(structured,Try3))
MergeLists.append(os.path.join(structured,Try4))
MergeLists.append(os.path.join(unstructured))

for currdir in MergeLists:
    print(os.path.exists(currdir))

True
True
True
True


In [15]:
modalityToLook = ["CT","PET"]
for currdir in MergeLists:
    PxList = np.sort(os.listdir(currdir))
    print("Number of Patients:",len(PxList)," Example: ",PxList[0])

Number of Patients: 664  Example:  0004879
Number of Patients: 339  Example:  0010539
Number of Patients: 30  Example:  0372344
Number of Patients: 1185  Example:  0004879


In [19]:
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Px','ACCT_Path','ACCT_bool','PET_Path','PET_bool',"Matching_Bool"])
    
    for currdir in MergeLists:
        PxList = np.sort(os.listdir(currdir))

        for Px in PxList:
            lastUID_CT = -1
            lastUID_PET = 0 
            ACCT_LD_fpaths = []
            PET_Earl_fpaths = []

            bool_PET = False
            bool_ACCT = False
            bool_match = False

            for root, dirs, files in os.walk(structured+Try2+Px, topdown=True):
                for d in dirs:
                    if "AC_CT_Body" in d or "CT van PET" in d or "CT LD" in d or "AC CT" in d or "AC  CT" in d:
                        ACCT_LD_fpaths.append(root+'/'+d+'/')
                        lastUID_CT = getICUID(root+'/'+d+'/')
                        bool_ACCT = True
                    if "PET" in d or "Earl" in d:
                        PET_Earl_fpaths.append(root+'/'+d+'/')
                        lastUID_PET = getICUID(root+'/'+d+'/')
                        bool_PET = True
                    if lastUID_CT == lastUID_PET:
                        bool_match = True 
                        break

            if bool_PET or bool_ACCT:
                writer.writerow([Px,ACCT_LD_fpaths,bool_ACCT,PET_Earl_fpaths,bool_PET,bool_match])
            
        